## 문서요약 fine-tuning하기
- 전체 문서의 내용을 입력으로 줬을때, `생성 요약문 작성`
- 법률 문서 : 법원 판결문 뉴스 텍스트 및 법원 주요 판결문 텍스트


### 실습 개요

1) 실습 목적
- 이번 실습에서는 GPT-2 모델을 PyTorch를 사용하여 구현하고 fine-tuning 해봅니다.
- GPT 모델은 decoder-only 구조를 통해, 자연어 처리에서 뛰어난 성능을 보여주는 모델으로 GPT 모델의 구조와 downstream task 에서의 fine-tuning 방식을 이해하고, 학습이 이뤄지는 과정을 학습해봅니다

2) 수강 목표
  * GPT 모델의 구조와 fine-tuning 방법을 이해한다
  * GPT 모델을 문서 요약을 위한 모델로 fine-tuning하고, 학습 과정에서 일어나는 연산과 input/output 형태에 대해 이해한다
  * GPT 모델을 원하는 downstream task로 tuning 하여 사용할 수 있다

#### 실습 목차

1. Dataset & Tokenizing
  * 1-1. summ_dataset class 정의
  * 1-2. load_Data, tokenized_dataset 함수 정의
  * 1-3. prepare_dataset 함수 정의
2. Model & Trainer
  * 2-1. compute_metrics 함수 정의 (rouge)
  * 2-2. load_tokenizer_and_model_for_train 함수 정의
  * 2-3. load_trainer_for_train 함수 정의
  * 2-4. train 함수 정의
  * 2-5. arguments 지정 및 학습 진행
3. Inference & Evaluation
  * 3-1. load_model_for_inference 함수 정의
  * 3-2. inference 함수 정의
  * 3-3. infer_and_eval 함수 정의

In [1]:
%pip install rouge_score
!wget –no-check-certificate 'https://docs.google.com/uc?export=download&id=14s5orP5j6nNOqdmGdy6DkdyhcIsn-6Zv' -O ./dataset/summarization//train.csv
!wget –no-check-certificate 'https://docs.google.com/uc?export=download&id=1wMhyqoJ0D7xQepW4y7Ireq2U2X61ndYz' -O ./dataset/summarization/test.csv

Note: you may need to restart the kernel to use updated packages.
--2024-08-29 14:25:07--  http://xn--no-check-certificate-2t2l/
Resolving xn--no-check-certificate-2t2l (xn--no-check-certificate-2t2l)... failed: Temporary failure in name resolution.
wget: unable to resolve host address ‘xn--no-check-certificate-2t2l’
--2024-08-29 14:25:07--  https://docs.google.com/uc?export=download&id=14s5orP5j6nNOqdmGdy6DkdyhcIsn-6Zv
Resolving docs.google.com (docs.google.com)... 172.217.161.206, 2404:6800:400a:80b::200e
Connecting to docs.google.com (docs.google.com)|172.217.161.206|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://drive.usercontent.google.com/download?id=14s5orP5j6nNOqdmGdy6DkdyhcIsn-6Zv&export=download [following]
--2024-08-29 14:25:08--  https://drive.usercontent.google.com/download?id=14s5orP5j6nNOqdmGdy6DkdyhcIsn-6Zv&export=download
Resolving drive.usercontent.google.com (drive.usercontent.google.com)... 142.250.206.193, 2404:6800:400a:

In [2]:
import os
import copy
import torch
import random
import evaluate
import datasets
import numpy as np
import pandas as pd
import pytorch_lightning as pl

from torch.utils.data import Dataset
from sklearn.model_selection import train_test_split
#from sklearn.metrics import accuracy_score, f1_score


from transformers import (
    AutoTokenizer,
    AutoConfig,
    AutoModelForCausalLM,
    GPT2LMHeadModel
)
from rouge_score import rouge_scorer, scoring
from transformers import EarlyStoppingCallback
from transformers.optimization import get_cosine_with_hard_restarts_schedule_with_warmup
from transformers import Trainer, TrainingArguments, Seq2SeqTrainer, Seq2SeqTrainingArguments

2024-08-29 14:25:15.709127: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-29 14:25:15.784892: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-08-29 14:25:16.170858: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:
2024-08-29 14:25:16.170911: W tensorflow/compiler/xl

In [3]:
class summ_dataset(Dataset):
    """dataframe을 torch dataset class로 변환"""
    def __init__(self, document, tokenizer):
      self.dataset = document
      self.tokenizer = tokenizer

    def __getitem__(self,idx):
        input_ids=torch.LongTensor(self.dataset["input_ids"][idx])
        labels=torch.LongTensor(self.dataset["labels"][idx])

        attention_mask=input_ids.ne(self.tokenizer.pad_token_id) ## padding token은 attention 계산에 반영되면 안되니까 mask를 정의한다..

        return dict(input_ids=input_ids, labels=labels, attention_mask=attention_mask)

    def __len__(self):
        return len(self.dataset["input_ids"])

#### 1-2.load_data , construct_tokenized_dataset 함수 정의

In [4]:
def load_data(dataset_dir):
    """csv file을 dataframe으로 load"""
    dataset = pd.read_csv(dataset_dir)
    return dataset


def tokenized_dataset(dataset, doc_tokenizer, sum_tokenizer, doc_max_length, sum_max_len, mode="train"):
    """
    토크나이징을 위한 함수. training과 inference 단계에서의 토크나이징이 별도로 구축되어 있다.
    - 학습일 때는 본문과 요약이 함께 입력된다. --> 본문 [SEP] 요약
    - 반면 추론 단계에서는 본문만 입력되어 요약을 생성해야함.
    """
    ## 추론 단계
    if mode == "infer":
      ## inference 시에는 document 만 주어지고, 마지막에 bos_token을 붙여 생성 시작하게 한다.
      document_text = dataset['document']
      summ_text = dataset['summary']

      ## document + bos
      ## <pad> <pad> d_1 d_2 d_3 ... d_n <bos>
      document = [doc_tokenizer(documents, padding = 'max_length', truncation=True, max_length=doc_max_length-1, add_special_tokens=True)['input_ids'] + [doc_tokenizer.bos_token_id] for documents in document_text.values]
      
      # labels에는 요약문만큼의 빈칸으로 채워준 후 모델이 예측하도록 함
      labels = [[-100] * sum_max_len for _ in document]

      out = {"input_ids": document, "labels": labels}
      print("inference을 위한 데이터에서 tokenizing 된 input 형태")
      print(document[-1])
      print(doc_tokenizer.convert_ids_to_tokens(document[-1]))
      print()

    elif mode == "train":
      document_text = dataset['document']
      summary_text = dataset['summary']
      ## document 와 summary를 이어 붙여서 모델 학습에 사용. 
      ## document 뒤에는 bos_token 을 붙여 생성 시작을 명시하고, summary 를 붙인 후 맨 뒤에는 eos_token 으로 생성의 끝을 명시.
      ## ⭐️ document를 padding 할 때는 side를 left로 주고, summary를 padding 할 때는 side를 right로 줘서 연속된 문장이 생성될 수 있도록 한다.
      ## ⭐️ <pad> <pad> d_1 d_2 d_3 ... d_n <bos> s_1 s_2 ... s_m <eos> <pad> <pad>
      document = [doc_tokenizer(documents, padding='max_length', truncation=True, max_length=doc_max_length-1, add_special_tokens=True)['input_ids'] + [doc_tokenizer.bos_token_id] for documents in document_text.values]
      summary = [sum_tokenizer(summaries + sum_tokenizer.eos_token, padding = 'max_length',truncation=True, max_length=sum_max_len, add_special_tokens=True)['input_ids'] for summaries in summary_text.values]

      ## 구성해둔 document 와 summary를 결합하여 input 준비
      tokenized_senetences = [document + summary for (document, summary) in zip(document, summary)]
      ## document는 생성할 내용이 아니므로 -100으로 label을 부여한다.
      # Input : <pad> <pad> d_1  d_2  d_3  ... d_n  <bos> s_1 s_2 ... s_m <eos> <pad> <pad>
      # Label : -100  -100    -100 -100 -100  ... -100  -100  s_1 s_2 ... s_m <eos> -100 -100

      labels = [[-100] * len(document) + summary for (document, summary) in zip(document, summary)]
      ## ⭐️ Q. 다음에 올 Token을 생성하도록 학습해야 되니까 s_1의 label은 한 칸씩 밀린 s_2가 들어가야 되지 않나요?
      # A. Transformer 라이브러리의 GPT 구현(https://github.com/huggingface/transformers/blob/main/src/transformers/models/gpt2/modeling_gpt2.py#L1103-L1104)을 보면, 
      # 모델의 Logit을 [: -1]만 가져오고, Label은 [1: ]을 가져와서 Loss를 계산하게 됩니다.
      # 즉, Input과 Label이 한 칸씩 밀린채로 입력을 넣지 않아도, 내부 구현에 의해 자동으로 밀린 채로 계산이 됩니다.

      # padding 된 부분이 학습되지 않도록 -100 으로 치환
      labels = [[-100 if token == sum_tokenizer.pad_token_id else token for token in l] for l in labels]
      out = {"input_ids": tokenized_senetences, "labels": labels}

      print("학습을 위한 데이터에서 tokenizing 된 input 형태")
      print(tokenized_senetences[-1])
      print(doc_tokenizer.convert_ids_to_tokens(tokenized_senetences[-1]))
      print()

      print("학습을 위한 데이터에서 label의 형태")
      print(labels[-1])
      print()

    return out

#### 1-3. prepare_dataset 함수 정의
- 앞서 정의한 함수들 기반으로 데이터셋 준비하는 함수

** 전처리과정 ** 
1. train.csv / test.csv 파일을 pd.dataframe 로 다운로드 해준다. <br>
2. train/validation set을 나눠준다. (7.5:2.5) <br>
3. 전체 문서와 요약 데이터를 모두 tokenizing 해준다.  <br>
4. 요약(label) 데이터는 Padding 된 부분은 loss 가 흐르지 않도록 -100 으로 치환해준다. <br>
5. tokenizing 된 데이터를 summ_dataset class로 반환해준다. <br>

In [5]:
def prepare_dataset(dataset_dir, doc_tokenizer,sum_tokenizer,doc_max_len, sum_max_len):
    """학습(train)과 평가(test)를 위한 데이터셋을 준비"""
    # load_data
    train_dataset = load_data(os.path.join(dataset_dir, "./dataset/summarization/train.csv"))
    test_dataset = load_data(os.path.join(dataset_dir, "./dataset/summarization/test.csv"))

    # split train / validation = 7.5 : 2.5
    train_dataset, val_dataset = train_test_split(train_dataset,test_size=0.2,random_state=42)

    ### tokenizer 에 들어가기 전 데이터 형태
    print("tokenizer 에 들어가는 데이터 형태")
    print(train_dataset.iloc[0])

    # tokenizing
    print("train tokenizing...")
    tokenized_train = tokenized_dataset(train_dataset, doc_tokenizer,sum_tokenizer, doc_max_len, sum_max_len)
    
    print("valid tokenizing...")
    tokenized_val = tokenized_dataset(val_dataset, doc_tokenizer,sum_tokenizer, doc_max_len, sum_max_len)
    
    print("test tokenizing...")
    tokenized_test = tokenized_dataset(test_dataset, doc_tokenizer,sum_tokenizer, doc_max_len, sum_max_len, mode="infer")

    # make dataset for pytorch.
    summ_train_dataset = summ_dataset(tokenized_train, doc_tokenizer)
    summ_val_dataset = summ_dataset(tokenized_val, doc_tokenizer)
    summ_test_dataset = summ_dataset(tokenized_test, doc_tokenizer)

    print("--- dataset class Done ---")

    return summ_train_dataset , summ_val_dataset, summ_test_dataset, test_dataset

### 2️⃣ Model & Trainer
- huggingface 에서 사전학습된(pre-trained) 모델을 불러옵니다.
- huggingface 의 Trainer 모듈을 정의하고 학습에 사용될 Arguments 들을 지정해줍니다.

#### 2-1. compute_metrics 함수 정의
- 학습 중 validation 할 때 사용될 평가지표 정의하는 함수
- 해당 실습에서는 Rouge Score를 Metric으로 사용

Q. Rouge Score?

문서 요약, 기계 번역 등의 task 에서 모델이 생성한 문장을 평가하기 위한 지표.

다양한 종류의 score 가 있지만, 이 중 ROUGE-N의 F1-score(recall 과 precisiond을 반영)를 사용합니다.

- ROUGE-N : N gram에 기반하여, 모델이 생성한 요약문과 정답 요약문 중 겹치는 단어의 개수를 확인하여 생성한 요약문을 평가하는 평가지표
- ROUGE-N recall : 두 요약문 중 겹치는 N-gram의 수 / 정답 요약문의 N-gram의 수
- ROUGE-N precision : 두 요약문 중 겹치는 N-gram의 수 / 모델이 생성한 요약문의 N-gram의 수  

예) ROUGE-1
- 모델이 생성한 요약문(R) : "the hello a cat dog fox jumps"
- 정답 요약문 (T) : "the fox jumps" 
- 두 요약문 중 겹치는 1-gram ['the', 'fox', 'jumps'] 이므로 ROUGE-1 recall 은 3/3, ROUGE-1 precision 은 3/7 = 0.43

In [6]:
def compute_metrics(args, pred):
    # tokenizer load
    MODEL_NAME = args.model_name
    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

    # 예측값과 정답
    labels = pred.label_ids
    preds  = pred.predictions.argmax(-1)
    if isinstance(preds, tuple):
      preds = preds[0]

    # preds에서 document 이후부터 생성된 summary를 decode
    ## GPT-2는 encoder-decoder 구조가 아니기 때문에 입력된 document에 이어서 summary를 생성하므로
    ## document 부분을 별도로 잘라내야 한다.
    decoded_preds = tokenizer.batch_decode(preds[:, args.doc_max_len:], skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels[:, args.doc_max_len:], skip_special_tokens=True)

    # rouge score 계산
    metric = datasets.load_metric("rouge")
    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)

    # ROUGE 결과를 추출
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    result = {k: round(v, 4) for k, v in result.items()}
    return {
        'Rouge-2' : result['rouge2']
        }

#### 2-2.load_tokenizer_and_model_for_train 함수 정의

In [7]:
def load_tokenizer_and_model_for_train(args):
    """학습(train)을 위한 사전학습(pretrained) 토크나이저와 모델을 huggingface에서 load"""
    # model과 tokenizer를 load
    MODEL_NAME = args.model_name
    doc_tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, padding_side="left")
    sum_tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, padding_side="right")

    # model의 hyperparameter를 setting
    model_config = AutoConfig.from_pretrained(MODEL_NAME)

    print(model_config)

    model = AutoModelForCausalLM.from_pretrained(MODEL_NAME, config=model_config)
    print("--- Modeling Done ---")
    
    return doc_tokenizer, sum_tokenizer, model

#### 2-3.load_trainer_for_train 함수 정의

In [8]:
def load_trainer_for_train(args, model, summ_train_dataset, summ_val_dataset):
    """학습(train)을 위한 huggingface trainer 설정"""
    
    training_args = TrainingArguments(
        output_dir=args.save_path + "results",  # output directory
        save_total_limit=args.save_limit,  # number of total save model.
        save_steps=args.save_step,  # model saving step.
        num_train_epochs=args.epochs,  # total number of training epochs
        learning_rate=args.lr,  # learning_rate
        per_device_train_batch_size=args.batch_size,  # batch size per device during training
        per_device_eval_batch_size=1,  # batch size for evaluation
        warmup_steps=args.warmup_steps,  # number of warmup steps for learning rate scheduler
        weight_decay=args.weight_decay,  # strength of weight decay
        logging_dir=args.save_path + "logs",  # directory for storing logs
        logging_steps=args.logging_steps,  # log saving step.
        evaluation_strategy="steps",  # evaluation strategy to adopt during training
            # `no`: No evaluation during training.
            # `steps`: Evaluate every `eval_steps`.
            # `epoch`: Evaluate every end of epoch.
        eval_steps=args.eval_steps,  # evaluation step.
        load_best_model_at_end=True,
    )

    ## Add callback & optimizer & scheduler
    MyCallback = EarlyStoppingCallback(
        early_stopping_patience=5, early_stopping_threshold=0.001
    )

    optimizer = torch.optim.AdamW(
        model.parameters(),
        lr=args.lr,
        betas=(0.9, 0.999),
        eps=1e-08,
        weight_decay=args.weight_decay,
        amsgrad=False,
    )

    print("--- Set training arguments Done ---")
    
    trainer = Trainer(
        model=model,  # the instantiated 🤗 Transformers model to be trained
        args=training_args,  # training arguments, defined above
        train_dataset=summ_train_dataset,  # training dataset
        eval_dataset=summ_val_dataset,  # evaluation dataset
        compute_metrics=lambda p: compute_metrics(args, p),
        callbacks=[MyCallback],
        optimizers=(
            optimizer,
            get_cosine_with_hard_restarts_schedule_with_warmup(
                    optimizer,
                    num_warmup_steps=args.warmup_steps,
                    num_training_steps=len(summ_train_dataset) * args.epochs,
            ),
        ),
    )
    
    print("--- Set Trainer Done ---")

    return trainer


#### 2-4.train 함수 정의

** 학습동작과정 ** 
1. 실험에 영향을 주는 모든 seed를 고정해준다. <br>
2. 사용할 gpu를 device에 할당해준다. <br>
3. tokenizer와 model을 불러온후, model을 device에 할당해준다. <br>
4. 학습에 사용될 summ_dataset 을 불러온다.<br>
5. 학습에 사용될 Trainer 를 불러온다.<br>
6. 학습을 진행한후에 best_model을 저장해준다. <br>

In [9]:
def train(args):
    """모델을 학습(train)하고 best model을 저장"""
    # fix a seed
    pl.seed_everything(seed=42, workers=False)

    # set device
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print("device:", device)

    # set model and tokenizer
    doc_tokenizer, sum_tokenizer , model = load_tokenizer_and_model_for_train(args)
    model.to(device)

    # set data
    summ_train_dataset, summ_val_dataset, summ_test_dataset, test_dataset = prepare_dataset(args.dataset_dir,doc_tokenizer, sum_tokenizer,args.doc_max_len,args.sum_max_len)
    # set trainer
    trainer = load_trainer_for_train(args, model, summ_train_dataset, summ_val_dataset)

    # train model
    print("--- Start train ---")
    trainer.train()
    print("--- Finish train ---")
    model.save_pretrained("./best_model")


#### 2-5.arguments 지정 및 학습 진행

In [10]:
class args():
    """학습(train)과 추론(infer)에 사용되는 arguments 관리하는 class"""
    dataset_dir = "./"
    model_type = "gpt2"
    model_name = 'MrBananaHuman/kogpt2_small'
    save_path = "./"
    save_step = 400
    logging_steps = 200
    eval_steps = 200
    save_limit = 5
    seed = 42
    epochs = 20 # 10
    batch_size = 4  # 메모리 상황에 맞게 조절 e.g) 16 or 32
    doc_max_len = 196
    sum_max_len = 64
    lr = 3e-5
    weight_decay = 0.01
    warmup_steps = 5
    scheduler = "linear"
    model_dir = "./best_model" #추론 시, 저장된 모델 불러오는 경로 설정
    
train(args)

Seed set to 42


device: cuda:0
GPT2Config {
  "_name_or_path": "MrBananaHuman/kogpt2_small",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "transformers_version": "4.42.3",
  "use_cache": true,
  "vocab_size": 52000
}



/home/pervinco/miniconda3/envs/upstage/lib/python3.8/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


--- Modeling Done ---
tokenizer 에 들어가는 데이터 형태
document    [1] 甲이 토지소유자 乙에게서 토지를 임차한 후 주유소 영업을 위하여 지하에 유류...
summary     토지에 매설된 유류저장조는 토지와 일체를 이루는 구성 부분이 아니므로 토지 임차인이...
Name: 249, dtype: object
train tokenizing...
학습을 위한 데이터에서 tokenizing 된 input 형태
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 12712, 600, 979, 723, 1042, 545, 3772, 1314, 13479, 12712, 1450, 1171, 7832, 979, 135, 3500, 723, 1254, 23745, 11328, 12712, 600, 979, 7303, 1172, 4658, 1036, 476, 752, 2653, 12712, 600, 979, 1667, 16747, 7832, 694, 5242, 27950, 10509, 7491, 5208, 10703, 2867, 1377, 386, 8459, 7309, 8065, 1104, 319, 5295, 14660, 12712, 600, 979, 1042, 545, 16798, 11242, 1487, 10093, 1042, 545, 2336, 1681, 1249, 5490, 1377, 386, 8459, 6650, 549, 1673, 2792, 11242, 1487, 10093, 1042, 545, 2823, 18961, 12712, 600, 29958, 56

/home/pervinco/.local/lib/python3.8/site-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Step,Training Loss,Validation Loss,Rouge-2
200,2.525000,2.182662,13.071000
400,1.458100,2.210392,13.871200
600,0.844200,2.292986,14.362700
800,0.460400,2.376865,15.302400
1000,0.246700,2.460108,12.715500
1200,0.146400,2.499961,13.264300
1400,0.098100,2.561069,15.711100


/tmp/ipykernel_235648/2187569660.py:20: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = datasets.load_metric("rouge")


There were missing keys in the checkpoint model loaded: ['lm_head.weight'].


--- Finish train ---


: 